In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import FastICA

from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, cross_val_score, train_test_split

from hyperopt import fmin, tpe, hp
from bayes_opt import BayesianOptimization

# SVR optimization methods

In [2]:
def bayesian_optimization_svr(X, y, cv=6, max_iter_svr=100, max_iter_opt=15):
    svr_opt = BayesianOptimization(
        lambda epsilon: cross_val_score(
            LinearSVR(C=100, epsilon=epsilon, max_iter=max_iter_svr),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        {'epsilon': (0.0, 5.0)},
        verbose=0
    )
    
    svr_opt.init(10)
    svr_opt.maximize(n_iter=max_iter_opt)
    
    return svr_opt.res['max']['max_params']['epsilon']

def hyperopt_optimization_svr(X, y, cv=6, max_iter_svr=100, max_iter_opt=15):
    space = hp.choice('regressor_type', [
        {
            'type': 'svr',
            'epsilon': hp.uniform('svr_epsilon', 0.0, 5.0)
        }
    ])

    best = fmin(
        fn=lambda args: cross_val_score(
            LinearSVR(C=100, epsilon=args['epsilon'], max_iter=max_iter_svr),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        space=space,
        algo=tpe.suggest,
        max_evals=max_iter_opt
    )

    return best['svr_epsilon']

def evaulate_best_svr_argument(X, y, eps_vals: list, cv=6):
    scores = []
    for e in eps_vals:
        scores.append(cross_val_score(
            LinearSVR(C=100, epsilon=e, max_iter=200),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean())
        
    return eps_vals[np.argmax(scores)]

# RF regressor optimization code

In [3]:
# hyperopt for lightgbm shows terrible results
def hyperopt_optimization_lightgbm(X, y, cv=6, max_iter_opt=15):
    space = hp.choice('regressor_type', [
        {
            'type': 'lightgbm',
            'feature_fraction': hp.uniform('feature_fraction', 0.05, 0.95),
            'bagging_fraction': hp.uniform('bagging_fraction', 0.05, 0.95),
            'bagging_freq': hp.uniform('bagging_freq', 1, 50),
            'n_estimators': hp.uniform('n_estimators', 5, 50),
            #'max_bin': hp.uniform('max_bin', )
        }
    ])

    best = fmin(
        fn=lambda args: cross_val_score(
            LGBMRegressor(
                boosting_type='rf', 
                feature_fraction=args['feature_fraction'], 
                bagging_freq=int(args['bagging_freq']), 
                bagging_fraction=args['bagging_fraction'],
                n_estimators=int(args['n_estimators'])
            ),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        space=space,
        algo=tpe.suggest,
        max_evals=max_iter_opt
    )
    
    return best

def bayesian_optimization_lightgbm(X, y, cv=6, max_iter_opt=15):
    svr_opt = BayesianOptimization(
        lambda feature_fraction, bagging_freq, bagging_fraction, n_estimators: cross_val_score(
            LGBMRegressor(
                boosting_type='rf', 
                feature_fraction=feature_fraction, 
                bagging_freq=int(bagging_freq), 
                bagging_fraction=bagging_fraction,
                n_estimators=int(n_estimators)
            ),
            X, y.squeeze(), cv=KFold(n_splits=cv).split(X), scoring='neg_mean_squared_error'
        ).mean(),
        {'feature_fraction': (0.05, 0.95),
         'bagging_fraction': (0.05, 0.95),
         'bagging_freq': (1, 50),
         'n_estimators': (5, 50) },
        verbose=0
    )
    
    svr_opt.init(10)
    svr_opt.maximize(n_iter=max_iter_opt)
    
    return svr_opt.res['max']['max_params']#['C']

## Facebook Comment Volume Dataset

In [4]:
df = pd.read_csv('datasets/facebook_comments.csv', index_col=0)

In [5]:
print('dataset size: {}'.format(df.shape))

dataset size: (40949, 54)


In [6]:
# encoding categorical variables
# H Local - category
# Post Promotion Status - category
# Base Time - time variable
# Page Category - category

H_Local = OneHotEncoder().fit_transform(df['H Local'].values.reshape(-1, 1)).todense()
Post_Promotion_Status = OneHotEncoder().fit_transform(df['Post Promotion Status'].values.reshape(-1, 1)).todense()
Base_Time = OneHotEncoder().fit_transform(df['Base Time'].values.reshape(-1, 1)).todense()
Page_Category = OneHotEncoder().fit_transform(df['Page Category'].values.reshape(-1, 1)).todense()

In [7]:
y = df.Target.values.reshape(-1, 1)
X = df.drop(['H Local', 'Post Promotion Status', 'Base Time', 'Page Category', 'Target'], axis=1).values
X = np.hstack([X, H_Local, Post_Promotion_Status, Base_Time, Page_Category])

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [9]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (40949, 228)


### Training model with ICA-processed data

In [12]:
# number of components is fixed to 10
ica = FastICA(n_components=10)
X_ica = ica.fit_transform(X_train)
X_val_ica = ica.transform(X_val)

In [13]:
X_ica, y_train = shuffle(X_ica, y_train)

### * Estimating SVR penalty

In [14]:
eps_hyperopt = hyperopt_optimization_svr(X_ica, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)
eps_bayesian = bayesian_optimization_svr(X_ica, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-3.6177188]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 57, 'nit': 7, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.14115973]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 55, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-9.50210452]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 2, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:335: UserWa

In [15]:
eps_opt = evaulate_best_svr_argument(X_ica, y_train, [eps_hyperopt, eps_bayesian], cv=6)

### * Estimating LightGBM params

In [16]:
params_opt = bayesian_optimization_lightgbm(X_train, y_train, cv=6, max_iter_opt=50)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.42612155]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 49, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-1.62693076]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 48, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-1.55897449]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserW

In [17]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

### * Evaluate with CV

In [18]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=100, epsilon=eps_opt, max_iter=200),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

In [19]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_ica, y_train):
        X_crossval_train, X_crossval_test = X_ica[train_ix], X_ica[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        # here must be sume sort of optimization
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))

In [20]:
print('for decorrelated data using ICA (averate scores)')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using ICA (averate scores)
linear, MSE: 979.0974515809114
svr, MSE: 1078.214427530016
forest, MSE: 1301.8619306696535


In [23]:
print('for decorrelated data using ICA (validation scores):')
for name, model in models.items():
    model.fit(X_ica, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val_ica), y_val.squeeze())))

for decorrelated data using ICA (validation scores):
linear, MSE: 886.4990818365944
svr, MSE: 1107.2444114939578
forest, MSE: 1417.8398046398047


## Parkinsons Telemonitoring Data Set

In [52]:
df = pd.read_csv('datasets/parkinsons_updrs.data')

In [53]:
print('dataset size: {}'.format(df.shape))

dataset size: (5875, 22)


In [54]:
subject = df['subject#'].unique().tolist()
subject_binary = np.zeros((df.shape[0], len(subject)))
for k, i in df.iterrows():
    subject_binary[k, int(i['subject#']) - 1] = 1

In [55]:
# replacing categorical features with binary values
y = df.total_UPDRS.values
X = df.drop(['motor_UPDRS', 'total_UPDRS', 'subject#'], axis=1).values
X = np.concatenate([subject_binary, X], axis=1)

In [56]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [57]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (5875, 61)


### Training model with ICA-processed data

In [58]:
# number of components is fixed to 10
ica = FastICA(n_components=10)
X_ica = ica.fit_transform(X_train)
X_val_ica = ica.transform(X_val)

In [59]:
X_ica, y_train = shuffle(X_ica, y_train)

### * Estimating SVR penalty

In [60]:
eps_hyperopt = hyperopt_optimization_svr(X_ica, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)
eps_bayesian = bayesian_optimization_svr(X_ica, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.0097045]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 54, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.01129719]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 7, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnin

In [61]:
eps_opt = evaulate_best_svr_argument(X_ica, y_train, [eps_hyperopt, eps_bayesian], cv=6)

### * Estimating LightGBM params

In [62]:
params_opt = bayesian_optimization_lightgbm(X_train, y_train, cv=6, max_iter_opt=50)

D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.0016018]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 54, 'nit': 2, 'warnflag': 2}
  " state: %s" % convergence_dict)
D:\Program Files (x86)\Anaconda\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-3.12001425e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 54, 'nit': 2, 'warnflag': 2}
  " state: %s" % convergence_dict)


In [63]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

### * Evaluate with CV

In [64]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=100, epsilon=eps_opt, max_iter=200),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

In [65]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_ica, y_train):
        X_crossval_train, X_crossval_test = X_ica[train_ix], X_ica[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        # here must be sume sort of optimization
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))

In [66]:
print('for decorrelated data using ICA')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using ICA
linear, MSE: 81.75051216352603
svr, MSE: 82.46107464397488
forest, MSE: 956.0731875133065


In [67]:
print('for decorrelated data using ICA (validation scores):')
for name, model in models.items():
    model.fit(X_ica, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val_ica), y_val.squeeze())))

for decorrelated data using ICA (validation scores):
linear, MSE: 85.0417709422462
svr, MSE: 85.71758733269266
forest, MSE: 961.1154826647958


## Energy efficiency Data Set

In [68]:
df = pd.read_excel('./datasets/ENB2012_data.xlsx')

In [69]:
print('dataset size: {}'.format(df.shape))

dataset size: (768, 10)


In [70]:
subject = df['X6'].unique().tolist()
subject_map = dict(zip(subject, range(len(subject))))
subject_binary = np.zeros((df.shape[0], len(subject)))
for k, i in df.iterrows():
    subject_binary[k, subject_map[i['X6']]] = 1

In [71]:
# replacing categorical features with binary values
y = df.Y1.values
X = df.drop(['Y1', 'Y2', 'X6'], axis=1).values
X = np.concatenate([subject_binary, X], axis=1)

In [72]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [73]:
print('dataset size after preprocessing: {}'.format(X.shape))

dataset size after preprocessing: (768, 11)


### Training model with ICA-processed data

In [76]:
# number of components is fixed to 5
ica = FastICA(n_components=5)
X_ica = ica.fit_transform(X_train)
X_val_ica = ica.transform(X_val)

In [77]:
X_ica, y_train = shuffle(X_ica, y_train)

### * Estimating SVR penalty

In [78]:
eps_hyperopt = hyperopt_optimization_svr(X_ica, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)
eps_bayesian = bayesian_optimization_svr(X_ica, y_train, cv=4, max_iter_svr=200, max_iter_opt=15)

In [79]:
eps_opt = evaulate_best_svr_argument(X_ica, y_train, [eps_hyperopt, eps_bayesian], cv=6)

### * Estimating LightGBM params

In [80]:
params_opt = bayesian_optimization_lightgbm(X_train, y_train, cv=6, max_iter_opt=50)

In [81]:
params_opt['bagging_freq'] = int(params_opt['bagging_freq'])
params_opt['n_estimators'] = int(params_opt['n_estimators'])

### * Evaluate with CV

In [82]:
models = {
    'linear': LinearRegression(),
    'svr': LinearSVR(C=100, epsilon=eps_opt, max_iter=200),
    'forest': LGBMRegressor(boosting_type='rf', **params_opt)
}

In [83]:
stats = {}

for k, model in models.items():
    stats[k] = []
    kfold = KFold(n_splits=8, shuffle=True)
    
    for train_ix, test_ix in kfold.split(X_ica, y_train):
        X_crossval_train, X_crossval_test = X_ica[train_ix], X_ica[test_ix]
        y_crossval_train, y_crossval_test = y_train[train_ix], y_train[test_ix]
        
        # here must be sume sort of optimization
        model.fit(X_crossval_train, y_crossval_train.ravel())
        stats[k].append(mean_squared_error(model.predict(X_crossval_test), y_crossval_test))

In [84]:
print('for decorrelated data using ICA')
for model, model_stats in stats.items():
    print('{}, MSE: {}'.format(model, np.mean(model_stats)))

for decorrelated data using ICA
linear, MSE: 21.519151782115316
svr, MSE: 21.998777446472516
forest, MSE: 598.1371824865678


In [85]:
print('for decorrelated data using ICA (validation scores):')
for name, model in models.items():
    model.fit(X_train, y_train.squeeze())
    print('{}, MSE: {}'.format(name, mean_squared_error(model.predict(X_val), y_val.squeeze())))

for decorrelated data using ICA (validation scores):
linear, MSE: 116.15323392774
svr, MSE: 172.61538442556483
forest, MSE: 118.17767193979121
